### Libraries 

In [1]:
%matplotlib inline
from keras.applications.imagenet_utils import preprocess_input
from keras.backend.tensorflow_backend import set_session
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Concatenate
import numpy as np
from scipy.misc import imread
import tensorflow as tf
import pickle
import keras
from ssd import SSD300
from ssd_training import MultiboxLoss
from ssd_utils import BBoxUtility
from random import shuffle
import matplotlib.pyplot as plt
from SSD_tester import calc_detection_prec_rec, calc_detection_ap


config = tf.ConfigProto(
    gpu_options=tf.GPUOptions(
        allow_growth=True # True->必要になったら確保, False->全部
    )
)
sess = sess = tf.Session(config=config)


Using TensorFlow backend.


In [2]:
voc_classes = ['door_handle', 'book', 'bottle', 'pillow', 'bowl', 'phone', 'speaker', 'plate', 'table', 
               'cup', 'monitor', 'keyboard', 'letter_tray', 'sofa', 'paper_notebook', 'power_outlet', 
               'tabple_knife', 'soap', 'bookcase','chair']
NUM_CLASSES = len(voc_classes) + 1
input_shape = (300, 300, 3) #channel last

### Load the weight from the hdf5

In [3]:
model = SSD300(input_shape, num_classes=NUM_CLASSES)
model.load_weights('../checkpoints/VOCB3DO/weights', by_name=True)

(64,)


In [4]:
gt = pickle.load(open('../pkls/VOCB3DO.pkl', 'rb'))
keys = sorted(gt.keys())
num_train = int(round(0.8 * len(keys)))
train_keys = keys[:num_train]
val_keys = keys[num_train:]
num_val = len(val_keys)

path_prefix = '../dataset/VOCB3DO/KinectColor/'
inputs = []
images = []
val_keys = sorted(val_keys)
for key in val_keys:
    img_path = path_prefix + key
    img = image.load_img(img_path, target_size=(300, 300))
    img = image.img_to_array(img)
    images.append(imread(img_path))
    inputs.append(img.copy())
inputs = preprocess_input(np.array(inputs))


In [5]:
#print(inputs.shape)

In [6]:
priors = pickle.load(open('../pkls/prior_boxes_ssd300.pkl', 'rb'))
bbox_util = BBoxUtility(NUM_CLASSES, priors)
preds = model.predict(inputs, batch_size=32, verbose=1)
results = bbox_util.detection_out(preds, confidence_threshold=0.5)
results = np.array(results)

170/170 [==============================] - 8s     


In [7]:
gt_bboxes = []
gt_labels = []
gt_scores = []
for key in val_keys:
    index = np.where(gt[key][:, 4:] == 1)
    gt_bboxes.append(gt[key][:, :4])
    gt_labels.append((index[1]).reshape(len(index[1]), 1))
    gt_scores.append(np.ones((len(index[1]), 1)))
gt_bboxes = np.array(gt_bboxes)
gt_labels = np.array(gt_labels)
gt_scores = np.array(gt_scores)

In [8]:
pred_labels = []
pred_scores = []
pred_bboxes = []
for result in results:
    if len(result) != 0:
        nm = len(result[:, 1])
        pred_labels.append((result[:, 0]-1).reshape(nm, 1))
        pred_scores.append(result[:, 1:2].reshape(nm, 1))
        pred_bboxes.append(result[:, 2:].reshape(nm, 4))
    else:
        pred_labels.append(np.array([]).reshape(0, 1))
        pred_scores.append(np.array([]).reshape(0, 1))
        pred_bboxes.append(np.array([]).reshape(0, 1))
pred_labels = np.array(pred_labels)
pred_scores = np.array(pred_scores)

pred_bboxes = np.array(pred_bboxes)

In [9]:
print(gt_labels[0].shape)

(4, 1)


In [10]:
prec, rec = calc_detection_prec_rec(pred_labels, pred_scores, pred_bboxes, gt_bboxes, gt_labels, iou_thresh=0.3)
print(prec)

[array([ 1.]), array([ 1.        ,  1.        ,  0.66666667,  0.75      ,  0.6       ,
        0.66666667,  0.57142857,  0.5       ,  0.44444444,  0.5       ,
        0.45454545,  0.41666667,  0.46153846,  0.5       ,  0.46666667,
        0.5       ,  0.47058824,  0.5       ,  0.52631579,  0.5       ,
        0.52380952,  0.5       ,  0.47826087,  0.45833333,  0.48      ,
        0.5       ,  0.48148148,  0.46428571,  0.44827586,  0.43333333,
        0.4516129 ,  0.46875   ,  0.45454545,  0.44117647,  0.42857143,
        0.44444444,  0.43243243,  0.42105263,  0.41025641,  0.4       ,
        0.41463415]), array([ 0.        ,  0.5       ,  0.33333333,  0.5       ,  0.6       ,
        0.66666667,  0.71428571,  0.75      ,  0.77777778,  0.8       ,
        0.81818182,  0.83333333,  0.76923077,  0.78571429,  0.73333333,
        0.75      ,  0.70588235,  0.72222222,  0.73684211,  0.7       ,
        0.66666667,  0.63636364,  0.65217391,  0.625     ,  0.6       ,
        0.61538462,  0.6296

In [12]:
ap = calc_detection_ap(prec, rec, use_07_metric=True)
{'ap': ap, 'map': np.nanmean(ap)}

{'ap': array([ 0.09090909,  0.22700691,  0.35860864,  0.16363636,  0.38766234,
         0.        ,  0.        ,         nan,  0.25617741,  0.28704974,
         0.25974026,  0.        ,  0.        ,  0.26623377,  0.        ,
         0.02272727,  0.        ,  0.        ,  0.09090909,  0.44658245]),
 'map': 0.15038122820375407}

In [ ]:
for i, img in enumerate(images):
    # Parse the outputs.
    det_label = results[i][:, 0]
    det_conf = results[i][:, 1]
    det_xmin = results[i][:, 2]
    det_ymin = results[i][:, 3]
    det_xmax = results[i][:, 4]
    det_ymax = results[i][:, 5]

    # Get detections with confidence higher than 0.6.
    top_indices = [i for i, conf in enumerate(det_conf) if conf >= 0.5]

    top_conf = det_conf[top_indices]
    top_label_indices = det_label[top_indices].tolist()
    top_xmin = det_xmin[top_indices]
    top_ymin = det_ymin[top_indices]
    top_xmax = det_xmax[top_indices]
    top_ymax = det_ymax[top_indices]

    colors = plt.cm.hsv(np.linspace(0, 1, NUM_CLASSES)).tolist()
    print(img.shape)
    plt.imshow(img / 255.)
    currentAxis = plt.gca()

    for i in range(top_conf.shape[0]):
        xmin = int(round(top_xmin[i] * 300))
        ymin = int(round(top_ymin[i] * 300))
        xmax = int(round(top_xmax[i] * 300))
        ymax = int(round(top_ymax[i] * 300))
        score = top_conf[i]
        label = int(top_label_indices[i])
        label_name = voc_classes[label - 1]
        display_txt = '{:0.2f}, {}'.format(score, label_name)
        coords = (xmin, ymin), xmax-xmin+1, ymax-ymin+1
        color = colors[label]
        currentAxis.add_patch(plt.Rectangle(*coords, fill=False, edgecolor=color, linewidth=2))
        currentAxis.text(xmin, ymin, display_txt, bbox={'facecolor':color, 'alpha':0.5})
    
    plt.show()